<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/clasificaci%C3%B3n_de_im%C3%A1genes/clasificaci%C3%B3n_con_transformador_de_visi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de imágenes con Transformador de Visión (ViT)

**Descripción:** Implementación del modelo Vision Transformer (ViT) para  clasificación de imágenes.

## Introducción

En este ejemplo se implementa el Transformador de visión o [Vision Transformer (ViT)](https://arxiv.org/abs/2010.11929), modelo originalmente presentado por Alexey Dosovitskiy para la clasificación de imágenes. Para este ejemplo se utilizará el conjunto de datos CIFAR-100.

El modelo ViT aplica la arquitectura Transformer con atención propia (*self-attention*) a secuencias de parches de imagen, sin usar capas de convolución.

Este ejemplo requiere TensorFlow 2.4 o superior, así como
[Complementos de TensorFlow](https://www.tensorflow.org/addons/overview),
que se puede instalar usando el siguiente comando:


```python
pip install -U tensorflow-addons
```

## Librerías

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

## Preprocesamiento de datos

In [ ]:
num_classes = 100
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")


## Configuración de hiperparámetros

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
image_size = 72  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier


## Aumentación de datos

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)


## Perceptrón Multicapa (MLP)

In [ ]:

def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


## Creación de parches

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


Se muestran los parches para una imagen de muestra.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
image = x_train[np.random.choice(range(x_train.shape[0]))]
plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")

## Codificación de parches

La clase `PatchEncoder` transformará linealmente un parche proyectándolo en un vector de tamaño `projection_dim`. Además, agrega una una incrustación de posición aprendible en el vector proyectado.

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

## Construcción del modelo ViT

El modelo ViT consta de varios bloques de transformadores,
que utilizan la capa `layers.MultiHeadAttention` como mecanismo de autoatención
aplicado a la secuencia de parches. Los bloques Transformer producen un
tensor `[batch_size, num_patches,projection_dim]`, que se procesa a través de un
cabeza clasificadora con softmax para producir la salida final de probabilidades de clase.

A diferencia de la técnica descrita en el [artículo](https://arxiv.org/abs/2010.11929),
que antepone una incrustación aprendible a la secuencia de parches codificados para servir
como la representación de la imagen, todas las salidas del bloque Transformador final son
reformado con `layers.Flatten()` y usado como imagen
entrada de representación a la cabeza del clasificador.
Tenga en cuenta que la capa `layers.GlobalAveragePooling1D`
también podría usarse en su lugar para agregar las salidas del bloque Transformador,
especialmente cuando el número de parches y las dimensiones de proyección son grandes.

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


## Entrenamiento del modelo

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

Después de 100 épocas, el modelo ViT logra alrededor del 55% de *accuracy* y
82% de *accuracy* en el top 5 en los datos de prueba. Estos no son resultados competitivos en el conjunto de datos CIFAR-100, ya que un ResNet50V2 entrenado desde cero con los mismos datos puede lograr un *accuracy* del 67%.

Nótese que los resultados del estado del arte informados en el [artículo](https://arxiv.org/abs/2010.11929) se logran pre-entrenando el modelo ViT usando
el conjunto de datos JFT-300M, luego ajustándolo en el conjunto de datos de destino. Para mejorar la calidad del modelo.
sin entrenamiento previo, se puede intentar entrenar el modelo para más épocas, utilizando una mayor cantidad de capas Transformer, cambiando el tamaño de las imágenes de entrada, cambiando también el tamaño del parche o aumentando las dimensiones de proyección.

Además, como se menciona en el documento, la calidad del modelo se ve afectada no solo por las opciones de arquitectura, pero también por hiperparámetros como la tasa de aprendizaje, optimizador, decaimiento de peso, etc. En la práctica, se recomienda afinar un modelo ViT que fue pre-entrenado utilizando un gran conjunto de datos de alta resolución.